# Boolean Retrieval



## Getting Started

In this example, we will use the [Elaticsearch Python API](https://elasticsearch-py.readthedocs.io/en/master/). First, we will import and set-up all of the required Python modules and variables we will use later on. Additionally, if you wish to use `curl` instead of the Python API, the relevant command line function has been commented above each api request.

In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
es = Elasticsearch(urls=['localhost'], port=9200)

### Preliminaries -- Exercise 1

Note, you cannot proceed with the content on Boolean retrieval if you do not undertake this exercise.

Download the dataset at the [Queensland Art Gallery of Modern Art](https://data.qld.gov.au/dataset/qagoma-media-releases/resource/a1e4dffa-edb1-4e6d-a4a0-353aca79e9a3). Then create an index called `goma`, and import the dataset at `./goma.json`. The solution code is provided just below. Instead of executing the code below, try to create your own code.



******************


#### Exercise 1 -- Solution code

Create the index:

In [2]:
indexName = "goma"
if not es.indices.exists(indexName):
    print ("creating ", indexName, " index")
    res = es.indices.create(index=indexName)
    print(" response: '%s'" % res)
    


creating  goma  index
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'goma'}'


Now import the data in the dataset

In [ ]:
import json

with open("./feed.json", "r") as f:
    d = json.load(f)
    for item in d["qagoma_events"]:
        es.index(index="goma", body=item, doc_type="_doc")

********************

## Representing Boolean Expressions

Let's first investigate how to construct Boolean queries in Elasticsearch.

In popular Boolean-like search engines like PubMed, Boolean queries are expressed as a series of nested expressions, where keywords or subqueries are separated by an operator. As a trivial example, let's consider the following Boolean query:

`(a AND b AND (c OR d))`

One would expect this query to retrieve all of the documents that contain `a` and `b`, and also either/both `c` or `d`. When we place our operators in between operands, we call this an "infix" expression. You should be familiar with this type of expression representation as it is how math expressions are taught in school.

Elasticsearch and it's DSL, on the otherhand, place the operators before the operands (a "prefix" expression). If we were to rearrange the previous expression to our new representation, we get:

`(AND a b (OR c d))`

Now, let's explore how to construct the same query in Elasticsearch as a JSON formatted expression:

In [ ]:
example_query = \
{
    'query': {
        'bool': {
            'must': [
                {
                    'match': {
                     'field': 'a'   
                    }
                },
                {
                    'match': {
                     'field': 'b'   
                    }
                },
                {
                    'bool': {
                        'should': [
                            {
                                'match': {
                                    'field': 'c'
                                }
                            },
                            {
                                'match': {
                                    'field': 'd'
                                }
                            }
                        ]
                    }
                }
            ]
        }
    }
}

The DSL of Elasticsearch is obviously more verbose than our trivial example, and you should take note of several things:

 1. Elasticsearch does not use `AND`, `OR`, etc. and instead calls these operads `must`, `filter`, `should`, and, `must_not`: see https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html#query-dsl-bool-query for a complete reference.
 2. The Boolean query in Elasticsearch (`bool`) is a compound query. Any type of query can be nested inside a `bool` query, including other `bool` queries.

## Searching with Boolean Queries

Let's explore the Boolean query API of Elasticsearch with some examples. First, let's visually see the effect `must` and `should` have on the search listing.

In [ ]:
must_query = \
{
    'query': {
        'bool': {
            'must': [
                {
                    'match': {
                        'description': 'art'                        
                    }
                },
                {
                    'match': {
                        'description': 'australian'
                    }
                }
            ]
        }
    }
}

should_query = \
{
    'query': {
        'bool': {
            'should': [
                {
                    'match': {
                        'description': 'art'                        
                    }
                },
                {
                    'match': {
                        'description': 'australian'
                    }
                }
            ]
        }
    }
}

# curl -X GET localhost:9200/goma/_search -d @must_query.json
must_res = es.search(index='goma', body=must_query)
should_res = es.search(index='goma', body=should_query)

pd.DataFrame([must_res['hits']['total'], should_res['hits']['total']], 
             index=['must', 'should'],
             columns=['Number of results'])

The `must` query is much more restrictive than the `should` query, as documents are required to contain both terms being matched (`must` = `AND`), rather than one or both (`should` = `OR`).

### Exercise 2

Now, attempt to write and execute other Boolean queries. Consider for example the following Boolean queries (also, attempt to formulate your own queries to explore Elasticsearch Boolean operators. For each query, write the corresponding Elasticsearch query and execute it against the `goma` index.

1. africa AND art
2. NOT australian
3. (art OR australian) NOT (art AND australian)


# Gotchas -- Coordination level Matching with `multi_match` and `match_phrase`. 

(Depending on time, you may want to do this section as homework)

You may be tempted to use a `multi_match` query instead of several `match` queries for the same query (for example for the query `art OR australian`) but on different fields in your `bool` query. This however creates a coordination-level matching expression: this means documents are scored (and ranked). In particular,  Elasticsearch will take the maximum score from the `multi_match` and sum the scores of the queries inside one of the `bool` queries. Another possibility is to use the `match_phrase`. This will produce two different rankings:

The `multi_match`:

```
{
    "query": {
        "bool": {
            "must": [
                {
                    "multi_match": {
                        "query": "a",
                        "fields": [
                            "abstract",
                            "title"
                        ],
                        "type": "phrase"
                    }
                }
            ]
        }
    }
}
```

orders as:
```
{
  "took": 5,
  "timed_out": false,
  "_shards": {
    "total": 5,
    "successful": 5,
    "failed": 0
  },
  "hits": {
    "total": 4,
    "max_score": 0.2876821,
    "hits": [
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1duSV_gZ47e2uMXwv2",
        "_score": 0.2876821,
        "_source": {
          "title": "a b c",
          "abstract": "a"
        }
      },
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1durdPgZ47e2uMXwv3",
        "_score": 0.2876821,
        "_source": {
          "title": "a",
          "abstract": "a b c"
        }
      },
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1dusyTgZ47e2uMXwv4",
        "_score": 0.2876821,
        "_source": {
          "title": "d c",
          "abstract": "a"
        }
      },
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1duuKPgZ47e2uMXwv5",
        "_score": 0.25811607,
        "_source": {
          "title": "a b",
          "abstract": "a b"
        }
      }
    ]
  }
}
```

And the two `match_phrase`s:

```
{
    "query": {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {
                                "match_phrase": {
                                    "abstract": "a"
                                }
                            },
                            {
                                "match_phrase": {
                                    "title": "a"
                                }
                            }
                        ]
                    }
                }
            ]
        }
    }
}
```

orders as:
```
{
  "took": 3,
  "timed_out": false,
  "_shards": {
    "total": 5,
    "successful": 5,
    "failed": 0
  },
  "hits": {
    "total": 4,
    "max_score": 0.5408423,
    "hits": [
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1duSV_gZ47e2uMXwv2",
        "_score": 0.5408423,
        "_source": {
          "title": "a b c",
          "abstract": "a"
        }
      },
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1durdPgZ47e2uMXwv3",
        "_score": 0.5408423,
        "_source": {
          "title": "a",
          "abstract": "a b c"
        }
      },
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1duuKPgZ47e2uMXwv5",
        "_score": 0.51623213,
        "_source": {
          "title": "a b",
          "abstract": "a b"
        }
      },
      {
        "_index": "trec-test",
        "_type": "article",
        "_id": "AV1dusyTgZ47e2uMXwv4",
        "_score": 0.2876821,
        "_source": {
          "title": "d c",
          "abstract": "a"
        }
      }
    ]
  }
}
```

The `multi_match` phrase query expands into a `dis_max`:
https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-dis-max-query.html, whereas the boolean query is summing the scores of both `should` queries.